# Lane Detection


---

**The tools you have are color selection, region of interest selection, grayscaling, Gaussian smoothing, Canny Edge Detection and Hough Tranform line detection.  You  are also free to explore and try other techniques that were not presented in the lesson.  Your goal is piece together a pipeline to detect the line segments in the image, then average/extrapolate them and draw them onto the image for display (as below).  Once you have a working pipeline, try it out on the video stream below.**

---

<figure>
 <img src="examples/line-segments-example.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above) after detecting line segments using the helper functions below </p> 
 </figcaption>
</figure>
 <p></p> 
<figure>
 <img src="examples/laneLines_thirdPass.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your goal is to connect/average/extrapolate line segments to get output like this</p> 
 </figcaption>
</figure>

### Ideas for Lane Detection Pipeline
**Some Core OpenCV functions that might be useful for this project are:**<br />
`cv2.Canny()` for Canny edge detection<br />
`cv2.HoughLinesP` for Hough Line Detection<br />

**Some other OpenCV functions that might be useful for this project are:**<br />
`cv2.inRange()` for color selection  <br />
`cv2.fillPoly()` for regions selection  <br />
`cv2.line()` to draw lines on an image given endpoints  <br />
`cv2.addWeighted()` to coadd / overlay two images<br />
`cv2.cvtColor()` to grayscale or change color<br />
`cv2.imwrite()` to output images to file  <br />
`cv2.bitwise_and()` to apply a mask to an image<br />

**Check out the OpenCV documentation to learn about these and discover even more awesome functionality!**

## 0.Import Packages (Modules)
---


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import math
import numpy as np
import cv2
%matplotlib inline

## 1.Read in an Image
---

In [ ]:
#reading in an image
image_name = 'solidWhiteRight.jpg'
image = mpimg.imread('test_images/'+image_name)

# image = cv2.imread('test_images/solidWhiteRight.jpg')

#printing out some stats and plotting
print('This image is:', type(image), 'with dimensions:', image.shape)
plt.imshow(image)  # if you wanted to show a single color channel image called 'gray', for example, call as plt.imshow(gray, cmap='gray')

## 2.Grayscale Image
[cv2.Canny()](https://docs.opencv.org/3.1.0/da/d22/tutorial_py_canny.html) for Canny edge detection<br />

In [ ]:
def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)


In [ ]:
gray_image = grayscale(image)
print('This is gray image:', type(gray_image), 'with dimensions:', gray_image.shape)
plt.imshow(gray_image, cmap='gray')

## 3. Edge Detection
[cv2.Canny()](https://docs.opencv.org/3.1.0/da/d22/tutorial_py_canny.html) for Canny edge detection<br />
[cv2.GaussianBlur](https://docs.opencv.org/3.1.0/d4/d13/tutorial_py_filtering.html) for Gaussian Blur<br />



In [ ]:
def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)


def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

In [ ]:
# do Gaussian Blur on gray image
kernel_size = 5
blur_gray = gaussian_blur(gray_image, kernel_size)

# do Canny edge detection
low_threshold = 50
high_threshold = 150
edges = canny(blur_gray, low_threshold, high_threshold)


print('This is detected edges with dimensions:', edges.shape)
# plt.imshow(blur_gray, cmap='gray')
plt.imshow(edges, cmap='gray')

## 4.Create Masks (Region of Interest)
[cv2.fillPoly()](https://docs.opencv.org/3.0-beta/modules/imgproc/doc/drawing_functions.html) for Canny edge detection<br />

In [ ]:
def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    mask = cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image, mask


In [ ]:
# Create mask image using cv2.fillPoly()
imshape = image.shape
vertices = np.array([[(0,imshape[0]),(int(imshape[1]/2-50),int(imshape[0]/2+60)), (int(imshape[1]/2+50),int(imshape[0]/2+60)), (imshape[1],imshape[0])]], dtype=np.int32)
masked_edges, mask = region_of_interest(edges, vertices)

print('This is the mask image with dimensions:', masked_edges.shape)
# plt.imshow(mask, cmap='gray')
plt.imshow(masked_edges, cmap='gray')


## 5.Houghline Transformation
[cv2.HoughLinesP](https://docs.opencv.org/3.0-beta/doc/py_tutorials/py_imgproc/py_houghlines/py_houghlines.html) for Hough Line Detection<br />

In [ ]:
def draw_lines_polyfit(img,lines,color=[255,0,0],thickness=3):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane.  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    leftline = [[],[]]
    rightline = [[],[]]
    frame_top = int(img.shape[0]/2+60)
    frame_bottom = int(img.shape[0])
    
    for line in lines:
        for x1,y1,x2,y2 in line:
#             cv2.line(img, (x1, y1), (x2, y2), color, thickness)
            slope_temp = slope(x1,y1,x2,y2)
            
            if slope_temp < -0.5 and slope_temp > -0.8:
                leftline[0].append(x1)
                leftline[0].append(x2)
                leftline[1].append(y1)
                leftline[1].append(y2)

            elif slope_temp > 0.5 and slope_temp < 0.8:
                rightline[0].append(x1)
                rightline[0].append(x2)
                rightline[1].append(y1)
                rightline[1].append(y2)
                
    y1 = frame_bottom
    y2 = frame_top          
    if len(leftline[0])>=2:
   
        leftline_fit = np.polyfit(leftline[0],leftline[1],1)
        x1_left = int((y1-leftline_fit[1])/leftline_fit[0])
        x2_left = int((y2-leftline_fit[1])/leftline_fit[0])
        cv2.line(img, (x1_left, y1), (x2_left, y2), color, thickness)

    if len(rightline[0])>=2:
        rightline_fit = np.polyfit(rightline[0],rightline[1],1)
        x1_right = int((y1-rightline_fit[1])/rightline_fit[0])
        x2_right = int((y2-rightline_fit[1])/rightline_fit[0])
        cv2.line(img, (x1_right, y1), (x2_right, y2), color, thickness)
                
def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)
            

    
def slope(x1,y1,x2,y2):
    return (y2-y1)/(x2-x1)
    
    
def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

def hough_lines_extrapolate(img, rho, theta, threshold, min_line_len, max_line_gap):
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines_polyfit(line_img, lines)
    return line_img


# Python 3 has support for cool math symbols.
def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

In [ ]:
rho = 1 #distance resolution in pixels of the Hough grid
theta = np.pi/180 #angular resolution in radians of hough grid
threshold = 12 #minimum number of votes
min_line_length = 12 #minimum number of pixels making up a line
max_line_gap = 2 #max number of pixels between two line segment
    
line_image = np.copy(image)*0 #creating a blank to draw lines on


# color_edges = hough_lines(masked_edges, rho, theta, threshold, min_line_length, max_line_gap)
    
color_edges = hough_lines_extrapolate(masked_edges, rho, theta, threshold, min_line_length, max_line_gap)

result_image = weighted_img(color_edges, image, α=0.7, β=1., λ=0.)
#     result_image = color_edges
plt.imshow(result_image)
plt.imsave('test_images_output/'+image_name, result_image)

## 6.Final Pipeline
Build the pipeline and run your solution on all test_images. Make copies into the `test_images_output` directory, and you can use the images in your writeup report.

Try tuning the various parameters, especially the low and high Canny thresholds as well as the Hough lines parameters.

In [ ]:
import os
image_names = os.listdir("test_images/")
print(image_names)


In [ ]:
# TODO: Build your pipeline that will draw lane lines on the test_images

# Loop through images
for i in range(len(image_names)):
    image_name = image_names[i]
    image = mpimg.imread('test_images/'+image_name)
    
    # make gray image
    gray = grayscale(image)

    # do Gaussian Blur on gray image
    kernel_size = 5
    blur_gray = gaussian_blur(gray, kernel_size)
    
    # do Canny edge detection
    low_threshold = 50
    high_threshold = 150
    edges = canny(blur_gray, low_threshold, high_threshold)

    # Create mask image using cv2.fillPoly()
    imshape = image.shape
    vertices = np.array([[(0,imshape[0]),(int(imshape[1]/2-50),int(imshape[0]/2+60)), (int(imshape[1]/2+50),int(imshape[0]/2+60)), (imshape[1],imshape[0])]], dtype=np.int32)
    masked_edges, mask = region_of_interest(edges, vertices)

    # Define the Hough transform parameters
    # Make a blank the same size as our image to draw on
    
    rho = 1 #distance resolution in pixels of the Hough grid
    theta = np.pi/180 #angular resolution in radians of hough grid
    threshold = 12 #minimum number of votes
    min_line_length = 12 #minimum number of pixels making up a line
    max_line_gap = 2 #max number of pixels between two line segment
    
    line_image = np.copy(image)*0 #creating a blank to draw lines on

#     color_edges = hough_lines(masked_edges, rho, theta, threshold, min_line_length, max_line_gap)
    
    color_edges = hough_lines_extrapolate(masked_edges, rho, theta, threshold, min_line_length, max_line_gap)

    result_image = weighted_img(color_edges, image, α=0.7, β=1., λ=0.)
#     result_image = color_edges
    # plt.imshow(image)
    plt.imsave('test_images_output/'+image_name,result_image)

print ("Saved")

## 7.Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

**Note: if you get an import error when you run the next cell, try changing your kernel (select the Kernel menu above --> Change Kernel). Still have problems? Try relaunching Jupyter Notebook from the terminal prompt. Also, consult the forums for more troubleshooting tips.**

**If you get an error that looks like this:**
```
NeedDownloadError: Need ffmpeg exe. 
You can download it by calling: 
imageio.plugins.ffmpeg.download()
```
**Follow the instructions in the error message and check out [this forum post](https://discussions.udacity.com/t/project-error-of-test-on-videos/274082) for more troubleshooting tips across operating systems.**

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML


In [ ]:
def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image where lines are drawn on lanes)

    # make gray image
    gray = grayscale(image)

    # do Gaussian Blur on gray image
    kernel_size = 5
    blur_gray = gaussian_blur(gray,kernel_size)

    # do Canny edge detection
    low_threshold = 50
    high_threshold = 150
    edges = canny(blur_gray, low_threshold, high_threshold)

    # Create mask image using cv2.fillPoly()
    imshape = image.shape
    vertices = np.array([[(0,imshape[0]),(int(imshape[1]/2-50),int(imshape[0]/2+60)), (int(imshape[1]/2+50),int(imshape[0]/2+60)), (imshape[1],imshape[0])]], dtype=np.int32)
    masked_edges, mask = region_of_interest(edges, vertices)
    
    # Define the Hough transform parameters
    # Make a blank the same size as our image to draw on
    rho = 1
    theta = np.pi/180
    threshold = 12
    min_line_length = 12
    max_line_gap = 2
    line_image = np.copy(image)*0 #creating a blank to draw lines on

    color_edges = hough_lines_extrapolate(masked_edges, rho, theta, threshold, min_line_length, max_line_gap)
    result = weighted_img(color_edges, image, α=0.6, β=1., λ=0.)
    
    return result

Let's try the one with the solid white lane on the right first ...

In [ ]:
white_output = 'test_videos_output/solidWhiteRight.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds


clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,2)

# clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")

white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!

%time white_clip.write_videofile(white_output, audio=False)

Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [ ]:
yellow_output = 'test_videos_output/solidYellowLeft.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4').subclip(0,5)
clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))